In [ ]:
# Daum 뉴스 크롤링 하기

In [20]:
import requests
import lxml.html
import pandas as pd
from pandas.io import sql
import os
import sqlite3

import re
import string

In [24]:
page =1
REG_DATE = 20210717
max_page =0

In [28]:
def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text:
            body.append(re.sub(punc, ' ', p.text))
    full_body = ' '.join(body)
    return full_body

def db_select():
    with sqlite3.connect(os.path.join('.', 'newDB')) as con:
        try:
            query = 'SELECT * FROM new'
            new = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return new

def db_save(LIST):
    with sqlite3.connect(os.path.join('.', 'newDB')) as con:
        try:
            LIST.to_sql(name = 'new', con = con, index = False, if_exists = 'append')
        except Exception as e :
            print(str(e))
        print(len(LIST), '건 저장')

while True:
    response = requests.get('https://news.daum.net/breakingnews/politics?page={}&regDate={}'.format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    df_list = []
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({
            'URL' : [url],
            'TITLE' : [a.text],
            'ARTICLE' : [article]
        })
        df_list.append(df)

    if df_list:
        df_t = pd.concat(df_list)
    db_save(df_t)

    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page <= num:
                max_page = num
        except:
            pass

    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')
    if (len(span) <=0) & (max_page < page):
        break
    else:
        page = page +1
    time.sleep(1)

11 건 저장


In [32]:
print(db_select())

                                       URL  \
0   https://v.daum.net/v/20210717001551118   
1   https://v.daum.net/v/20210717001502116   
2   https://v.daum.net/v/20210717001501115   
3   https://v.daum.net/v/20210717001004089   
4   https://v.daum.net/v/20210717000606041   
5   https://v.daum.net/v/20210717000547030   
6   https://v.daum.net/v/20210717000305000   
7   https://v.daum.net/v/20210717000111965   
8   https://v.daum.net/v/20210717000110963   
9   https://v.daum.net/v/20210717000052945   
10  https://v.daum.net/v/20210717000035940   

                                       TITLE  \
0     이재명 "공공기관 직원 특공 폐지해야..실질적 공평 이루는 게 중요"   
1            윤석열, 오늘 광주 방문.."5·18은 헌법 수호 항거"   
2             국회, 제73주년 제헌절 경축식..사상 최초 영상 진행   
3   '집단감염' 청해부대 전원 수송기로 조기 귀국..오늘 오전 추가검사 결과   
4          유승민 "진정한 민주공화국 아직도 요원..나라 새로 세워야"   
5            파병부대 백신 미접종 비판 잇따르자..정부 "추후 접종"   
6      '최재형 입당'에 정청래 "이준석, 윤석열에 한 방 먹여..잘했다"   
7        [취임 한달 톺아보기] 이슈 선점 탁월 이준석은 '양날의 칼?'   
8              